<div>
<img src="./images/sunpy_logo.png" width="500" align="left"/>
</div>

# 1. Searching and downloading data with sunpy



In this notebook, an introduction to how you can search for and download data with sunpy. We will begin with an intoduction to `astropy.units` (which are used throughout the sunpy ecosystem), and then look about how to use `Fido` and build queries for data. In particular, this notebook will look at the following:

1. Introduction to `astropy.units`
2. Overview of `Fido` 
3. Constructing a data search query and inspecting it
4. More complex queries and the HEK
5. Extending Fido - the SOAR archive

In [1]:
import astropy.units as u

from sunpy.net import Fido, attrs as a
from sunpy.time import parse_time

# sunpy_soar is an affiliated package of SunPy 
# and registers the SOAR to be searched by Fido
import sunpy_soar

import numpy as np

## 1.1 Astropy Units - a quick overview
[`astropy.units`](https://docs.astropy.org/en/stable/units/) provides a means to deal with and handle numbers/arrays etc that have an associated physical quantity (e.g. km, seconds, Kelvin). Throughout SunPy, any physical input or outputs is an [`astropy.Quantity`](https://docs.astropy.org/en/stable/units/quantity.html#quantity). Lets look at how we can create and convert between astropy units. Above we have imported `astropy.units` as `u`

In [2]:
distance_in_km = 10*u.km

In [3]:
distance_in_km

<Quantity 10. km>

In [4]:
distance_in_km.unit

Unit("km")

In [5]:
distance_in_km.value

10.0

We can convert between equivalent units

In [6]:
distance_in_km.cgs

<Quantity 1000000. cm>

In [7]:
distance_in_km.to(u.parsec)

<Quantity 3.24077929e-13 pc>

In [8]:
distance_in_km.to(u.Mm)

<Quantity 0.01 Mm>

However you can only convert between physical units that make sense for example:

In [9]:
#distance_in_km.to(u.second)

In [10]:
time_in_sec = 60*u.s

In [11]:
(distance_in_km/time_in_sec).unit

Unit("km / s")

In [12]:
(10*u.Angstrom).to(u.nm)

<Quantity 1. nm>

# 1.2 Overview of sunpy's Fido Unified Downloader
Fido is sunpy's interface for searching and downloading solar physics data.
It offers a unified interface for searching and fetching data irrespective of the underlying client or webservice from where the data is obtained.
You can also search and accesses multiple instruments and all available data providers in a single query.
It supplies a single, easy, consistent and extendable way to get most forms of solar physics data the community need.

For more information about Fido and how to use it check out the documentation on our website: https://docs.sunpy.org/en/stable/tutorial/acquiring_data/index.html

Fido offers access to data available through:

* VSO (Virtual Solar Observatory)
* JSOC (through drms)
* Individual data providers from web accessible sources (http, ftp, etc)
* CDAWeb
* HEK
* HELIO
  
As described here Fido provides access to many sources of data through different clients, these clients can be defined inside sunpy or in other packages (e.g. DKIST data can be accessed using Fido through [DKIST User Tools](https://docs.dkist.nso.edu/projects/python-tools/en/latest/tutorial/2_search_and_asdf_download.html)).


#### Importantly, Solar Orbiter data can be accessed through the client defined in the `sunpy_soar` affiliated package.
The SOAR client is registered once we install `sunpy_soar` above. Without installing it, it wont be registered within Fido.

Lets first inspect the clients that are available through Fido:

In [13]:
Fido

Client,Description
CDAWEBClient,Provides access to query and download from the Coordinated Data Analysis Web (CDAWeb).
EVEClient,Provides access to Level 0CS Extreme ultraviolet Variability Experiment (EVE) data.
GBMClient,Provides access to data from the Gamma-Ray Burst Monitor (GBM) instrument on board the Fermi satellite.
XRSClient,Provides access to several GOES XRS files archive.
SUVIClient,Provides access to data from the GOES Solar Ultraviolet Imager (SUVI).
GONGClient,Provides access to the Magnetogram products of NSO-GONG synoptic Maps.
LYRAClient,Provides access to the LYRA/Proba2 data archive.
NOAAIndicesClient,Provides access to the NOAA solar cycle indices.
NOAAPredictClient,Provides access to the NOAA SWPC predicted sunspot Number and 10.7 cm radio flux values.
SRSClient,Provides access to the NOAA SWPC solar region summary data.


### Using attributes to search for data with Fido

Sunpy uses specified **attributes** to search for data using Fido. The range of these attributes is located in the `attrs` submodule. These `attr` parameters can be combined together to construct data search queries, such as searching over a certain time period, for data from a certain instrument with a certain wavelength etc.

Different clients and provides will have client-specific attributes, but the core attributes are:

* `a.Time`
* `a.Instrument`
* `a.Wavelength`


Lets look at how these attributes work in more detail.

First we can look at `a.Time`, which is used to specify the timerange of a query.

In [46]:
a.Time("2022-04-02 12:00", "2022-04-02 15:00")

<sunpy.net.attrs.Time(2022-04-02 12:00:00.000, 2022-04-02 15:00:00.000)>

We can inspect the instrument attribute to see what instrument `attrs` are currently supported through sunpy. Here we can see the instrument name (i.e. the name to be passed to the `a.Instrument` attribute, the client from which the data is available to access, and the full name of the instrument.)

In [47]:
a.Instrument

Attribute Name,Client,Full Name,Description
aia,VSO,AIA,Atmospheric Imaging Assembly
bcs,VSO,BCS,Bragg Crystal Spectrometer
be_continuum,VSO,BE-Continuum,INAF-OACT Barra Equatoriale Continuum Instrument
be_halpha,VSO,BE-Halpha,INAF-OACT Barra Equatoriale Hα Instrument
bigbear,VSO,Big Bear,"Big Bear Solar Observatory, California TON and GONG+ sites"
caii,VSO,CAII,Kanzelhöhe Ca II k Instrument
cds,VSO,CDS,Coronal Diagnostic Spectrometer
celias,VSO,CELIAS,"Charge, Element, and Isotope Analysis System"
cerrotololo,VSO,Cerro Tololo,"Cerro Tololo, Chile GONG+ site"
chp,VSO,chp,Chromospheric Helium-I Imaging Photometer


sunpy also now provides tab completion to auto-fill the attribute name

In [48]:
a.Instrument.eit

<sunpy.net.attrs.Instrument(EIT: Extreme ultraviolet Imaging Telescope) object at 0x1236e43e0>

To search for certain wavelengths, we need to specify the input as an `astropy.Quantity` which is a the combination of a value and an associated unit. This is something is universal in the sunpy stack - that every physical input/output is a `Quantity`.

In [49]:
a.Wavelength(17.1*u.angstrom)

<sunpy.net.attrs.Wavelength(17.1, 17.1, 'Angstrom')>

## 3. Constructing a search query
 ### A simple query

Lets create a simple query to search for data from AIA over a particular time period

In [58]:
result = Fido.search(a.Time("2022-04-02 12:00", "2022-04-02 15:00"), 
                     a.Instrument("AIA"))

In [59]:
result

Start Time,End Time,Source,Instrument,Wavelength,Provider,Physobs,Wavetype,Extent Width,Extent Length,Extent Type,Size,Extra Flags
,,,,Angstrom,,,,,,,Mibyte,
Time,Time,str3,str3,float64[2],str4,str9,str6,str4,str4,str8,float64,str1
2022-04-02 12:00:00.000,2022-04-02 12:09:13.000,SDO,AIA,335.0 .. 335.0,JSOC,intensity,NARROW,4096,4096,FULLDISK,3038.47656,S
2022-04-02 12:00:04.000,2022-04-02 12:09:17.000,SDO,AIA,193.0 .. 193.0,JSOC,intensity,NARROW,4096,4096,FULLDISK,3038.47656,S
2022-04-02 12:00:05.000,2022-04-02 12:00:06.000,SDO,AIA,4500.0 .. 4500.0,JSOC,intensity,NARROW,4096,4096,FULLDISK,64.64844,S
2022-04-02 12:00:05.000,2022-04-02 12:09:18.000,SDO,AIA,304.0 .. 304.0,JSOC,intensity,NARROW,4096,4096,FULLDISK,3038.47656,S
2022-04-02 12:00:06.000,2022-04-02 12:09:19.000,SDO,AIA,131.0 .. 131.0,JSOC,intensity,NARROW,4096,4096,FULLDISK,3038.47656,S
...,...,...,...,...,...,...,...,...,...,...,...,...
2022-04-02 14:59:28.000,2022-04-02 14:59:53.000,SDO,AIA,193.0 .. 193.0,JSOC,intensity,NARROW,4096,4096,FULLDISK,193.94531,S
2022-04-02 14:59:29.000,2022-04-02 14:59:54.000,SDO,AIA,304.0 .. 304.0,JSOC,intensity,NARROW,4096,4096,FULLDISK,193.94531,S


Now lets make our query a bit more specific, say, say we only want one wavelength band from AIA. This can be achieved by specifying the `Wavelength` attribute within the search. The `a.Wavelength` attribute is passed as an `astropy.Quantity`:

In [60]:
result = Fido.search(a.Time("2022-04-02 12:00", "2022-04-02 15:00"), 
                     a.Instrument("AIA"), 
                     a.Wavelength(304*u.angstrom))

In [61]:
result

Start Time,End Time,Source,Instrument,Wavelength,Provider,Physobs,Wavetype,Extent Width,Extent Length,Extent Type,Size
,,,,Angstrom,,,,,,,Mibyte
Time,Time,str3,str3,float64[2],str4,str9,str6,str4,str4,str8,float64
2022-04-02 12:00:05.000,2022-04-02 12:00:06.000,SDO,AIA,304.0 .. 304.0,JSOC,intensity,NARROW,4096,4096,FULLDISK,64.64844
2022-04-02 12:00:17.000,2022-04-02 12:00:18.000,SDO,AIA,304.0 .. 304.0,JSOC,intensity,NARROW,4096,4096,FULLDISK,64.64844
2022-04-02 12:00:29.000,2022-04-02 12:00:30.000,SDO,AIA,304.0 .. 304.0,JSOC,intensity,NARROW,4096,4096,FULLDISK,64.64844
2022-04-02 12:00:41.000,2022-04-02 12:00:42.000,SDO,AIA,304.0 .. 304.0,JSOC,intensity,NARROW,4096,4096,FULLDISK,64.64844
2022-04-02 12:00:53.000,2022-04-02 12:00:54.000,SDO,AIA,304.0 .. 304.0,JSOC,intensity,NARROW,4096,4096,FULLDISK,64.64844
...,...,...,...,...,...,...,...,...,...,...,...
2022-04-02 14:59:05.000,2022-04-02 14:59:06.000,SDO,AIA,304.0 .. 304.0,JSOC,intensity,NARROW,4096,4096,FULLDISK,64.64844
2022-04-02 14:59:17.000,2022-04-02 14:59:18.000,SDO,AIA,304.0 .. 304.0,JSOC,intensity,NARROW,4096,4096,FULLDISK,64.64844


We can further specify this query by choosing the cadence (time-sampling) of the data we want to search for and download. This can be achieved by using the Sample attribute. Similar to the Wavelength attributes, this needs to be an astropy Quantity. Lets further specify the search above to only search for data with a cadence of 10 minutes.

In [92]:
result = Fido.search(a.Time("2022-04-02 12:00", "2022-04-02 15:00"), 
                     a.Instrument("AIA"), 
                     a.Wavelength(171*u.angstrom),
                     a.Sample(10*u.min))

In [93]:
len(result[0])

18

## 1.3 Downloading the data

Now we can show how data that is queried above can be downloaded. Once the data you have searched for (and filtered etc) is constructed into a query using `Fido.search`, you can then easily download them using `Fido.fetch`.

The data is downloaded via asynchronous and parallel download streams (via parfive), and also allows for failed data downloads to be recognized so that files can be re-requested if not downloaded.

Lets now look at how a `UnifiedResponse` from a `Fido.search` can be passed to `Fido.fetch` to download the data

In [94]:
files = Fido.fetch(result)

Files Downloaded:   0%|          | 0/18 [00:00<?, ?file/s]

5/0 files failed to download. Please check `.errors` for details


These files are downloaded to a local location set in the sunpy.config.file, which by default is ~/sunpy/data/. Fido.fetch returns a parfile.Results object which gives the path to where the files are downloaded to

In [95]:
print(files[0])

IndexError: list index out of range

You can also define what directory you want the files to be saved to by passing the directory path to the path keyword in Fido.fetch. For example, I want to download these files to a local directory `./AIA/<name_of_file>`

In [96]:
Fido.fetch(result, path="./{instrument}/{file}")

Files Downloaded:   0%|          | 0/18 [00:00<?, ?file/s]

aia_lev1_171a_2022_04_02t12_00_09_35z_image_lev1.fits:   0%|          | 0.00/12.0M [00:00<?, ?B/s]

aia_lev1_171a_2022_04_02t12_40_09_35z_image_lev1.fits:   0%|          | 0.00/12.0M [00:00<?, ?B/s]

aia_lev1_171a_2022_04_02t12_20_09_35z_image_lev1.fits:   0%|          | 0.00/12.0M [00:00<?, ?B/s]

aia_lev1_171a_2022_04_02t12_10_09_35z_image_lev1.fits:   0%|          | 0.00/12.0M [00:00<?, ?B/s]

...

aia_lev1_171a_2022_04_02t14_20_09_35z_image_lev1.fits:   0%|          | 0.00/12.0M [00:00<?, ?B/s]

aia_lev1_171a_2022_04_02t14_30_09_35z_image_lev1.fits:   0%|          | 0.00/12.0M [00:00<?, ?B/s]

aia_lev1_171a_2022_04_02t14_40_09_35z_image_lev1.fits:   0%|          | 0.00/12.0M [00:00<?, ?B/s]

aia_lev1_171a_2022_04_02t14_50_09_35z_image_lev1.fits:   0%|          | 0.00/12.0M [00:00<?, ?B/s]

['AIA/aia_lev1_171a_2022_04_02t12_00_09_35z_image_lev1.fits', 'AIA/aia_lev1_171a_2022_04_02t12_10_09_35z_image_lev1.fits', 'AIA/aia_lev1_171a_2022_04_02t12_20_09_35z_image_lev1.fits', 'AIA/aia_lev1_171a_2022_04_02t12_30_09_35z_image_lev1.fits', 'AIA/aia_lev1_171a_2022_04_02t12_40_09_35z_image_lev1.fits', 'AIA/aia_lev1_171a_2022_04_02t12_50_09_35z_image_lev1.fits', 'AIA/aia_lev1_171a_2022_04_02t13_00_09_35z_image_lev1.fits', 'AIA/aia_lev1_171a_2022_04_02t13_10_09_35z_image_lev1.fits', 'AIA/aia_lev1_171a_2022_04_02t13_20_09_35z_image_lev1.fits', 'AIA/aia_lev1_171a_2022_04_02t13_30_09_35z_image_lev1.fits', 'AIA/aia_lev1_171a_2022_04_02t13_40_09_35z_image_lev1.fits', 'AIA/aia_lev1_171a_2022_04_02t13_50_09_35z_image_lev1.fits', 'AIA/aia_lev1_171a_2022_04_02t14_00_09_35z_image_lev1.fits', 'AIA/aia_lev1_171a_2022_04_02t14_10_09_35z_image_lev1.fits', 'AIA/aia_lev1_171a_2022_04_02t14_20_09_35z_image_lev1.fits', 'AIA/aia_lev1_171a_2022_04_02t14_30_09_35z_image_lev1.fits', 'AIA/aia_lev1_171a_2022

## 1. 4 More complex queries

In addition to making a query to one client for one instrument, `Fido` allows the flexibility to search for data from multiple instruments, wavelengths, times etc, even when the data is being obtained through different clients.

This query can be constructed by using the pipe `|` operator, which joins queries together just like the OR operator.

Lets now make a query that searches for both GOES/XRS and AIA data over a particular time period

In [67]:
result = Fido.search(a.Time("2022-04-02 12:00", "2022-04-02 15:00"), 
                     a.Instrument.xrs  | (a.Instrument.aia & a.Wavelength(304*u.angstrom) & a.Sample(10*u.min)))

In [68]:
len(result)

2

In [69]:
result

<sunpy.net.fido_factory.UnifiedResponse object at 0x1243f4f50>
Results from 2 Providers:

4 Results from the XRSClient:
Source: <8: https://umbra.nascom.nasa.gov/goes/fits 
8-15: https://www.ncei.noaa.gov/data/goes-space-environment-monitor/access/science/ 
16-17: https://data.ngdc.noaa.gov/platforms/solar-space-observing-satellites/goes/

       Start Time               End Time        ... Provider Resolution
----------------------- ----------------------- ... -------- ----------
2022-04-02 00:00:00.000 2022-04-02 23:59:59.999 ...     NOAA      flx1s
2022-04-02 00:00:00.000 2022-04-02 23:59:59.999 ...     NOAA      avg1m
2022-04-02 00:00:00.000 2022-04-02 23:59:59.999 ...     NOAA      flx1s
2022-04-02 00:00:00.000 2022-04-02 23:59:59.999 ...     NOAA      avg1m

18 Results from the VSOClient:
Source: http://vso.stanford.edu/cgi-bin/search
Total estimated size: 1.22 Gbyte

       Start Time               End Time        Source ... Extent Type   Size  
                                                       ...              Mibyte 
----------------------- ----------------------- ------ ... ----------- --------
2022-04-02 12:00:05.000 2022-04-02 12:00:06.000    SDO ...    FULLDISK 64.64844
2022-04-02 12:10:05.000 2022-04-02 12:10:06.000    SDO ...    FULLDISK 64.64844
2022-04-02 12:20:05.000 2022-04-02 12:20:06.000    SDO ...    FULLDISK 64.64844
2022-04-02 12:30:05.000 2022-04-02 12:30:06.000    SDO ...    FULLDISK 64.64844
2022-04-02 12:40:05.000 2022-04-02 12:40:06.000    SDO ...    FULLDISK 64.64844
                    ...                     ...    ... ...         ...      ...
2022-04-02 14:00:05.000 2022-04-02 14:00:06.000    SDO ...    FULLDISK 64.64844
2022-04-02 14:10:05.000 2022-04-02 14:10:06.000    SDO ...    FULLDISK 64.64844
2022-04-02 14:20:05.000 2022-04-02 14:20:06.000    SDO ...    FULLDISK 64.64844
2022-04-02 14:30:05.000 2022-04-02 14:30:06.000    SDO ...    FULLDISK 64.64844
2022-04-02 14:40:05.000 2022-04-02 14:40:06.000    SDO ...    FULLDISK 64.64844
2022-04-02 14:50:05.000 2022-04-02 14:50:06.000    SDO ...    FULLDISK 64.64844
Length = 18 rows

In [71]:
result[0]

Start Time,End Time,Instrument,SatelliteNumber,Physobs,Source,Provider,Resolution
Time,Time,str3,int64,str10,str4,str4,str5
2022-04-02 00:00:00.000,2022-04-02 23:59:59.999,XRS,16,irradiance,GOES,NOAA,flx1s
2022-04-02 00:00:00.000,2022-04-02 23:59:59.999,XRS,16,irradiance,GOES,NOAA,avg1m
2022-04-02 00:00:00.000,2022-04-02 23:59:59.999,XRS,17,irradiance,GOES,NOAA,flx1s
2022-04-02 00:00:00.000,2022-04-02 23:59:59.999,XRS,17,irradiance,GOES,NOAA,avg1m


In [72]:
result.all_colnames

['End Time',
 'Instrument',
 'Physobs',
 'Provider',
 'Resolution',
 'SatelliteNumber',
 'Source',
 'Start Time',
 'url']

Lets download the GOES XRS data first

In [73]:
Fido.fetch(result[0, 0], path="./{instrument}/{file}")

Files Downloaded:   0%|          | 0/1 [00:00<?, ?file/s]

sci_xrsf-l2-flx1s_g16_d20220402_v2-2-0.nc:   0%|          | 0.00/5.19M [00:00<?, ?B/s]

['XRS/sci_xrsf-l2-flx1s_g16_d20220402_v2-2-0.nc']

Now lets say we only want to download one AIA file at a particular time, we can also search the table for certain conditions. Lets say we just want the file that closest to 2022-04-02 13:00. 

In [75]:
(np.abs(result[1]["Start Time"] - parse_time("2022-04-02 13:00"))).argmin()

6

In [76]:
Fido.fetch(result[1, 6], path="./")

Files Downloaded:   0%|          | 0/1 [00:00<?, ?file/s]

aia_lev1_304a_2022_04_02t13_00_05_13z_image_lev1.fits:   0%|          | 0.00/7.50M [00:00<?, ?B/s]

['aia_lev1_304a_2022_04_02t13_00_05_13z_image_lev1.fits']


## 1. 5 Using External Fido Clients 

Within `sunpy` core, we support a number of clients to common data providers. However, the `Fido` search interface is extensible such that external packages can write that their own clients that extend `Fido` in order to additional data sources. One such example is the `sunpy_soar` package which adds a client for the Solar Orbter Archive (SOAR).


## SOAR archive searching using sunpy!

In [77]:
import sunpy_soar

Note that after importing `sunpy_soar`, the SOAR is now listed as a client that `Fido` will search.

In [78]:
Fido

Client,Description
CDAWEBClient,Provides access to query and download from the Coordinated Data Analysis Web (CDAWeb).
EVEClient,Provides access to Level 0CS Extreme ultraviolet Variability Experiment (EVE) data.
GBMClient,Provides access to data from the Gamma-Ray Burst Monitor (GBM) instrument on board the Fermi satellite.
XRSClient,Provides access to several GOES XRS files archive.
SUVIClient,Provides access to data from the GOES Solar Ultraviolet Imager (SUVI).
GONGClient,Provides access to the Magnetogram products of NSO-GONG synoptic Maps.
LYRAClient,Provides access to the LYRA/Proba2 data archive.
NOAAIndicesClient,Provides access to the NOAA solar cycle indices.
NOAAPredictClient,Provides access to the NOAA SWPC predicted sunspot Number and 10.7 cm radio flux values.
SRSClient,Provides access to the NOAA SWPC solar region summary data.


In [81]:
eui_query = Fido.search(a.Time("2022-04-02 12:00", "2022-04-02 15:00"), 
                        a.soar.Product("EUI-FSI174-IMAGE"), 
                        a.Level(2))

In [82]:
eui_query

Instrument,Data product,Level,Start time,End time,Data item ID,Filename,Filesize,SOOP Name
,,,,,,,Mbyte,
str3,str16,str2,str23,str23,str43,str52,float64,str30
EUI,eui-fsi174-image,L2,2022-04-02 12:00:45.225,2022-04-02 12:00:55.225,solo_L2_eui-fsi174-image_20220402T120045225,solo_L2_eui-fsi174-image_20220402T120045225_V01.fits,5.219,R_SMALL_MRES_MCAD_AR-Long-Term
EUI,eui-fsi174-image,L2,2022-04-02 12:10:45.226,2022-04-02 12:10:55.226,solo_L2_eui-fsi174-image_20220402T121045226,solo_L2_eui-fsi174-image_20220402T121045226_V01.fits,5.138,R_SMALL_MRES_MCAD_AR-Long-Term
EUI,eui-fsi174-image,L2,2022-04-02 12:20:45.227,2022-04-02 12:20:55.227,solo_L2_eui-fsi174-image_20220402T122045227,solo_L2_eui-fsi174-image_20220402T122045227_V01.fits,5.216,R_SMALL_MRES_MCAD_AR-Long-Term
EUI,eui-fsi174-image,L2,2022-04-02 12:30:45.228,2022-04-02 12:30:55.228,solo_L2_eui-fsi174-image_20220402T123045228,solo_L2_eui-fsi174-image_20220402T123045228_V01.fits,5.213,R_SMALL_MRES_MCAD_AR-Long-Term
EUI,eui-fsi174-image,L2,2022-04-02 12:40:45.229,2022-04-02 12:40:55.229,solo_L2_eui-fsi174-image_20220402T124045229,solo_L2_eui-fsi174-image_20220402T124045229_V01.fits,5.141,R_SMALL_MRES_MCAD_AR-Long-Term
...,...,...,...,...,...,...,...,...
EUI,eui-fsi174-image,L2,2022-04-02 14:10:45.275,2022-04-02 14:10:55.275,solo_L2_eui-fsi174-image_20220402T141045275,solo_L2_eui-fsi174-image_20220402T141045275_V01.fits,5.103,R_SMALL_MRES_MCAD_AR-Long-Term
EUI,eui-fsi174-image,L2,2022-04-02 14:20:45.240,2022-04-02 14:20:55.240,solo_L2_eui-fsi174-image_20220402T142045240,solo_L2_eui-fsi174-image_20220402T142045240_V01.fits,5.187,R_SMALL_MRES_MCAD_AR-Long-Term


In [85]:
Fido.fetch(eui_query, path="./{instrument}/{file}")

Files Downloaded:   0%|          | 0/18 [00:00<?, ?file/s]

solo_L2_eui-fsi174-image_20220402T133045235_V01.fits:   0%|          | 0.00/5.19M [00:00<?, ?B/s]

['EUI/solo_L2_eui-fsi174-image_20220402T120045225_V01.fits', 'EUI/solo_L2_eui-fsi174-image_20220402T121045226_V01.fits', 'EUI/solo_L2_eui-fsi174-image_20220402T122045227_V01.fits', 'EUI/solo_L2_eui-fsi174-image_20220402T123045228_V01.fits', 'EUI/solo_L2_eui-fsi174-image_20220402T124045229_V01.fits', 'EUI/solo_L2_eui-fsi174-image_20220402T125045230_V01.fits', 'EUI/solo_L2_eui-fsi174-image_20220402T130045231_V01.fits', 'EUI/solo_L2_eui-fsi174-image_20220402T131045269_V01.fits', 'EUI/solo_L2_eui-fsi174-image_20220402T132045233_V01.fits', 'EUI/solo_L2_eui-fsi174-image_20220402T133045235_V01.fits', 'EUI/solo_L2_eui-fsi174-image_20220402T134045261_V01.fits', 'EUI/solo_L2_eui-fsi174-image_20220402T135045237_V01.fits', 'EUI/solo_L2_eui-fsi174-image_20220402T140045238_V01.fits', 'EUI/solo_L2_eui-fsi174-image_20220402T141045275_V01.fits', 'EUI/solo_L2_eui-fsi174-image_20220402T142045240_V01.fits', 'EUI/solo_L2_eui-fsi174-image_20220402T143045241_V01.fits', 'EUI/solo_L2_eui-fsi174-image_20220402T

We can also search for other data products, for example the Solar Orbiter MAG

In [86]:
mag_query = Fido.search(a.Time("2022-04-02", "2022-04-05"), 
                        a.soar.Product("MAG-RTN-NORMAL-1-MINUTE"), 
                        a.Level(2))

In [87]:
mag_query

Instrument,Data product,Level,Start time,End time,Data item ID,Filename,Filesize,SOOP Name
,,,,,,,Mbyte,
str3,str23,str2,str23,str23,str40,str48,float64,object
MAG,mag-rtn-normal-1-minute,L2,2022-04-02 00:00:00.000,2022-04-03 00:00:00.000,solo_L2_mag-rtn-normal-1-minute_20220402,solo_L2_mag-rtn-normal-1-minute_20220402_V01.cdf,0.032,None
MAG,mag-rtn-normal-1-minute,L2,2022-04-03 00:00:00.000,2022-04-04 00:00:00.000,solo_L2_mag-rtn-normal-1-minute_20220403,solo_L2_mag-rtn-normal-1-minute_20220403_V01.cdf,0.032,None
MAG,mag-rtn-normal-1-minute,L2,2022-04-04 00:00:00.000,2022-04-05 00:00:00.000,solo_L2_mag-rtn-normal-1-minute_20220404,solo_L2_mag-rtn-normal-1-minute_20220404_V01.cdf,0.033,None
MAG,mag-rtn-normal-1-minute,L2,2022-04-05 00:00:00.000,2022-04-06 00:00:00.000,solo_L2_mag-rtn-normal-1-minute_20220405,solo_L2_mag-rtn-normal-1-minute_20220405_V01.cdf,0.033,None


In [88]:
mag_files = Fido.fetch(mag_query, path="./{instrument}/{file}")

Files Downloaded:   0%|          | 0/4 [00:00<?, ?file/s]

solo_L2_mag-rtn-normal-1-minute_20220405_V01.cdf:   0%|          | 0.00/32.7k [00:00<?, ?B/s]

solo_L2_mag-rtn-normal-1-minute_20220404_V01.cdf:   0%|          | 0.00/32.6k [00:00<?, ?B/s]

solo_L2_mag-rtn-normal-1-minute_20220403_V01.cdf:   0%|          | 0.00/32.5k [00:00<?, ?B/s]

solo_L2_mag-rtn-normal-1-minute_20220402_V01.cdf:   0%|          | 0.00/32.3k [00:00<?, ?B/s]

In [89]:
mag_files

['MAG/solo_L2_mag-rtn-normal-1-minute_20220402_V01.cdf', 'MAG/solo_L2_mag-rtn-normal-1-minute_20220403_V01.cdf', 'MAG/solo_L2_mag-rtn-normal-1-minute_20220404_V01.cdf', 'MAG/solo_L2_mag-rtn-normal-1-minute_20220405_V01.cdf']

# Accessing data from the CDAWeb with sunpy - which is very helpful for in-situ data

There is also a CDAWeb client within sunpy. CDAWeb data can be accessed when the `cdaweb.Dataset` attribute is provided to the search.

The data available from the SOAR is also available from the CDAWeb. You may be used to working with this (especially if you mainly work with in-situ observations), so lets go through how the data can also be accessed this way. This is handy, as you can also access many other in-situ measurements from this too.

In [90]:
res_cdaw = Fido.search(a.Time("2022-04-02", "2022-04-05"), 
                       a.cdaweb.Dataset('SOLO_L2_MAG-RTN-NORMAL-1-MINUTE'))

In [91]:
res_cdaw

Dataset,Start time,End time,URL
str31,str23,str23,str144
SOLO_L2_MAG-RTN-NORMAL-1-MINUTE,2022-04-02 00:00:29.000,2022-04-02 23:59:30.000,https://cdaweb.gsfc.nasa.gov/sp_phys/data/solar-orbiter/mag/science/l2/rtn-normal-1-minute/2022/solo_l2_mag-rtn-normal-1-minute_20220402_v01.cdf
SOLO_L2_MAG-RTN-NORMAL-1-MINUTE,2022-04-03 00:00:29.000,2022-04-03 23:59:30.000,https://cdaweb.gsfc.nasa.gov/sp_phys/data/solar-orbiter/mag/science/l2/rtn-normal-1-minute/2022/solo_l2_mag-rtn-normal-1-minute_20220403_v01.cdf
SOLO_L2_MAG-RTN-NORMAL-1-MINUTE,2022-04-04 00:00:29.000,2022-04-04 23:59:30.000,https://cdaweb.gsfc.nasa.gov/sp_phys/data/solar-orbiter/mag/science/l2/rtn-normal-1-minute/2022/solo_l2_mag-rtn-normal-1-minute_20220404_v01.cdf


In [45]:
mag_cdaw_files = Fido.fetch(res_cdaw)

Files Downloaded:   0%|          | 0/1 [00:00<?, ?file/s]

solo_l2_mag-rtn-normal-1-minute_20220325_v01.cdf:   0%|          | 0.00/60.2k [00:00<?, ?B/s]